# Assignment-1: Transformations and representations

Team Name: Sentinels

Roll number: 2019101054 2019111002

# Instructions

- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. See `Set Up` for detailed step-by-step instructions about the installation setup.
- Save all your results in ```results/<question_number>/<sub_topic_number>/```
- The **References** section provides you with important resources to solve the assignment.
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. 
- Make sure your code is modular since you may need to reuse parts for future assignments.
- Answer the descriptive questions in your own words with context & clarity. Do not copy answers from online resources or lecture notes.
- The **deadline** for this assignment is on 11/09/2021 at 11:55pm. Please note that there will be no extensions.
- Plagiarism is **strictly prohibited**.


# Submission Instructions

1. Make sure your code runs without any errors after reinitializing the kernel and removing all saved variables.
2. After completing your code and saving your results, zip the folder with name as ``Team_<team_name>_MR2021_Assignment_<assignment_number>.zip``

# Set Up

We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. All assignments will be python based, hence familiarising yourself with Python is essential.


## Setting up Anaconda environment (Recommended)

1. Install Anaconda or Miniconda from [here](https://docs.conda.io/projects/conda/en/latest/user-guide/install/linux.html) depending on your requirements.
2. Now simply run `conda env create -f environment.yml` in the current folder to create an environment `mr_assignment1` (`environment.yml` can be found in `misc/`).
3. Activate it using `conda activate mr_assignment1`.

## Setting up Virtual environment using venv

You can also set up a virtual environment using venv

1. Run `sudo apt-get install python3-venv` from command line.
2. `python3 -m venv ~/virtual_env/mr_assignment1`. (you can set the environment path to anything)
3. `source ~/virtual_env/mr_assignment1/bin/activate`
4. `pip3 install -r requirements.txt` from the current folder (`requirements.txt` can be found in `misc/`).

In [1]:
import open3d as o3d
import numpy as np
from copy import deepcopy
from time import sleep
import math
from scipy.optimize import fsolve
from scipy.spatial.transform.rotation import Rotation
R = Rotation


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 1. Getting started with Open3D

Open3D is an open-source library that deals with 3D data, such as point clouds, mesh. We'll be using Open3D frequently as we work with point clouds. Let's start with something simple:

<img src="misc/bunny.jpg" alt="drawing" width="200"/>

1. Read the Stanford Bunny file (in `data/`) given to you and visualise it using Open3D.
2. Convert the mesh to a point cloud and change the colour of points.
3. Set a predefined viewing angle (using Open3D) for visualization and display the axes while plotting.
4. Scale, Transform, and Rotate the rabbit (visualise after each step).
5. Save the point cloud as bunny.pcd.

In [4]:
Mesh_colors = [1, 0.706, 0]
pcd_colors = [0, 0.0, 1]

path = "data/bunny.ply"


def get_axis(size, origin):
    return o3d.geometry.TriangleMesh.create_coordinate_frame(size, origin)


class Mesh:
    def __init__(self, path):
        self.mesh = o3d.io.read_triangle_mesh(path)

    def visualize(self):
        print("Visualizing ...")
        o3d.visualization.draw_geometries([self.mesh])

    def compute_vertex_normal(self):
        print("Computing normal and rendering it.")
        self.mesh.compute_vertex_normals()

    def other_info(self):
        print("Mesh is :")
        print(self.mesh)
        print("Mesh Vertices : ")
        print(np.asarray(self.mesh.vertices))
        print("Mesh Triangles : ")
        print(np.asarray(self.mesh.triangles))
        print("Vertex Normals : ")
        print(self.mesh.has_vertex_normals())
        print("Vertex Colours : ")
        print(self.mesh.has_vertex_colors())

    def paint_mesh(self):
        self.mesh.paint_uniform_color(Mesh_colors)

    def save_copy(self, new_path):
        o3d.io.write_triangle_mesh(new_path, self.mesh)


class Pointcloud:

    def __init__(self, path=path, color=True):
        mesh = Mesh(path).mesh
        self.pcd = mesh.sample_points_uniformly(400000)
        if color:
            self.paint_pcd()

    def visualize(self):
        print("Visualizing ...")
        o3d.visualization.draw_geometries([self.pcd])

    def other_info(self):
        print("Pcd is :")
        print(self.pcd)
        print("PCD points : ")
        print(np.asarray(self.pcd.points))
        print("PCD Normals : ")
        print(np.asarray(self.pcd.normals))

    def paint_pcd(self):
        self.pcd.paint_uniform_color(pcd_colors)

    def saving_copy_pcdfile(self, newpath):
        o3d.io.write_point_cloud(newpath, self.pcd)


def visualize_std(objs):
    o3d.visualization.draw_geometries(objs,
                                      zoom=1,
                                      front=[0.3, 0, 1],
                                      lookat=[0.2, 0.07, 0],
                                      up=[0, 1, 0],
                                      )


def task_1_1():
    # TASK 1
    # Read the Stanford Bunny file (in data/) given to you and visualise it using Open3D
    bmesh = Mesh(path)
    bmesh.other_info()
    bmesh.visualize()


def task_1_2():
    # TASK 2
    # Convert the mesh to a point cloud and change the colour of points.
    bpcd = Pointcloud(path, False)
    bpcd.visualize()
    bpcd.other_info()
    bpcd.paint_pcd()
    bpcd.visualize()


def task_1_3():
    bpcd = Pointcloud(path)
    axis = get_axis(0.05, [0, 0, 0])
    visualize_std([axis, bpcd.pcd])


def task_1_4():
    bpcd = Pointcloud(path)
    obj = bpcd.pcd
    axis = get_axis(0.05, [0, 0, 0])
    new_color = [0.50, 0.20, 0.10]

    # Scale
    S = 0.5
    scObj = deepcopy(obj).paint_uniform_color(
        new_color).scale(S, center=[0, 0, 0])
    scAxis = deepcopy(axis).scale(S, center=[0, 0, 0])
    visualize_std([scObj, scAxis, axis, obj])
    # o3d.visualization.draw_geometries([scObj, scAxis, axis, obj])

    # Transform
    T = [1.30, 0.0, 0.0]
    theta = (np.pi / 4, 0, np.pi / 4)

    rotMat = axis.get_rotation_matrix_from_xyz(theta)
    TrMat = np.r_[np.c_[rotMat, T], [[0, 0, 0, 1]]]

    trObj = deepcopy(obj).paint_uniform_color(new_color).transform(TrMat)
    trAxis = deepcopy(axis).transform(TrMat)
    visualize_std([trObj, trAxis, axis, obj])
    # o3d.visualization.draw_geometries([trObj, trAxis, axis, obj])

    # rotate
    roObj = deepcopy(obj).paint_uniform_color(new_color).rotate(rotMat)
    roAxis = deepcopy(axis).rotate(rotMat)
    visualize_std([roObj, roAxis, axis, obj])
    # o3d.visualization.draw_geometries([roObj, roAxis, axis, obj])


def task_1_5():
    o3d.io.write_point_cloud("./results/1/5/bunny.pcd", Pointcloud(path).pcd)


In [3]:
task_1_1()
task_1_2()
task_1_3()
task_1_4()
task_1_5()

Mesh is :
TriangleMesh with 35947 points and 69451 triangles.
Mesh Vertices : 
[[-0.0378297   0.12794     0.00447467]
 [-0.0447794   0.128887    0.00190497]
 [-0.0680095   0.151244    0.0371953 ]
 ...
 [-0.0704544   0.150585   -0.0434585 ]
 [-0.0310262   0.153728   -0.00354608]
 [-0.0400442   0.15362    -0.00816685]]
Mesh Triangles : 
[[20399 21215 21216]
 [14838  9280  9186]
 [ 5187 13433 16020]
 ...
 [17279 34909 17346]
 [17277 17346 34909]
 [17345 17346 17277]]
Vertex Normals : 
False
Vertex Colours : 
False
Visualizing ...
Visualizing ...
Pcd is :
PointCloud with 400000 points.
PCD points : 
[[-0.09234853  0.13198229  0.01729623]
 [-0.09236169  0.13204612  0.01758458]
 [-0.09240787  0.13189332  0.017444  ]
 ...
 [-0.066744    0.03714927  0.03428296]
 [-0.06675944  0.03699914  0.03380299]
 [-0.06671837  0.03745814  0.03475077]]
PCD Normals : 
[]
Visualizing ...


# 2. Transformations and representations

## a) Euler angles
1. Write a function that returns a rotation matrix given the angles $\alpha$, $\beta$, and $\gamma$ in radians (X-Y-Z)

2. Solve for angles using ```fsolve from scipy``` for three initializations of your choice and compare.
$$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 & 0.944799 \\0.21984631 & 0.96542533 & 0.14007684 \\
    -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] 
$$

$$N(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -0.173648178 &  0.984807753 \\0 & 0.984807753 & 0.173648178 \\
    -1 & 0 & 0\end{array}\right] 
$$

3. What is a Gimbal lock? 

4. Show an example where a Gimbal lock occurs and visualize the Gimbal lock on the given bunny point cloud. You have to show the above by **animation** (cube rotating along each axis one by one).
    - *Hint: Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by $30^{\circ}$ around a particular axis, do in increments of $5^{\circ}$ 6 times to make it look like an animation.*


In [7]:
# ////////////////////////////////////////////////////

M = np.array([
    [0.26200263, -0.19674724, 0.944799],
    [0.21984631, 0.96542533, 0.14007684],
    [-0.93969262, 0.17101007, 0.29619813]])
N = np.array([
    [0, -0.173648178,  0.984807753],
    [0, 0.984807753, 0.173648178],
    [-1, 0, 0]])

# //////////////////////////////////////////////////////

def Rotation_matrix(alpha, beta, gamma):
    RX, RY, RZ = np.zeros((3, 3)), np.zeros((3, 3)), np.zeros((3, 3))

    RX[0][0], RX[1][1], RX[1][2], RX[2][1], RX[2][2] = 1, np.cos(
        gamma), -np.sin(gamma), np.sin(gamma), np.cos(gamma)

    RY[0][0], RY[0][2], RY[1][1], RY[2][0], RY[2][2] = np.cos(
        beta), np.sin(beta), 1, -np.sin(beta), np.cos(beta)

    RZ[0][0], RZ[0][1], RZ[1][0], RZ[1][1], RZ[2][2] = np.cos(
        alpha), -np.sin(alpha), np.sin(alpha), np.cos(alpha), 1

    return RZ @ RY @ RX


def calculate(X, M):
    pi2 = np.pi/2

    def get_angles(M):
        if M[2][0] == -1:
            # val of alpha=0,beta =np.pi/2 , gamma= atan2(M[0][1],M[1][1]) JJ book
            C = math.atan2(M[0][1], M[1][1])
            return [0., pi2, C]

        elif M[2][0] == 1:
            # val of alpha=0,beta =-np.pi/2 , gamma= -atan2(M[0][1],M[1][1]) JJ book
            C = -math.atan2(M[0][1], M[1][1])
            return [0., -pi2, C]

        else:
            vcos = np.sqrt(M[0][0]**2 + M[1][0]**2)
            p = math.atan2(M[1][0]/vcos, M[0][0]/vcos)
            q = math.atan2(-M[2][0], vcos)
            r = math.atan2(M[2][1]/vcos, M[2][2]/vcos)
            return [p, q, r]
            # val of alpha = Atan2(r 21 /cβ, r 11 /cβ)
            # beta = Atan2( −r31 , sqrt(r11 ^ 2 + r21 ^ 2 ) )
            # gamma = Atan2(r 32 /cβ, r 33 /cβ)

    return np.array(X) - get_angles(M)


def find_angles(rot_matrix):
    print("\nThe  input matrix (alpha,beta,gamma) is : \n", rot_matrix)

    np.random.seed(0)
    I = np.random.rand(3, 3)
    r1, r2, r3 = I[0], I[1], I[2]

    rM1 = fsolve(calculate, r1, rot_matrix)
    rM2 = fsolve(calculate, r2, rot_matrix)
    rM3 = fsolve(calculate, r3, rot_matrix)

    print("\n Initialization 1 is : ", r1,
          "\n Result obtained is : ", rM1, "\n")
    print("\n Initialization 2 is : ", r2,
          "\n Result obtained is : ", rM2, "\n")
    print("\n Initialization 3 is : ", r3,
          "\n Result obtained is : ", rM3, "\n")

    return rM1, rM2, rM3


def task_2_a_1():
    alpha, beta, gamma = 0, 0, np.pi/2
    print("The Calculated Rotation Matrix for "
          "[", alpha, ", ", beta, ", ", gamma, "] is :")
    rot_mat = Rotation_matrix(alpha, beta, gamma)
    print(rot_mat)
    print("Real Rotation matrix:")
    print(Rotation.from_euler('xyz',[gamma,beta, alpha]).as_matrix())


def task_2_a_2():
    print("\n for M: ")
    print(M, "\n")

    a = find_angles(M)
    for angles in a:
        print("expected angles: ", angles)
        print("expected rot_matrix:")
        print(Rotation_matrix(angles[0], angles[1], angles[2]))

    print("\n\n\n for N: ")
    print(N, "\n")

    a = find_angles(N)
    for angles in a:
        print("expected angles: ", angles)
        print("expected rot_matrix:")
        print(Rotation_matrix(angles[0], angles[1], angles[2]))


def task_2_a_4():
    axis = get_axis(0.05, [0, 0, 0])
    org_axis = deepcopy(axis)
    axis.translate([0.1, 0, 0])
    pcd = Pointcloud().pcd.translate([0, 0.1, 0])
    alpha = (-0/180)*np.pi
    beta = np.pi/2
    del_gamma = (np.pi/180)*(1)

    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(org_axis)
    vis.add_geometry(axis)
    vis.add_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()
    sleep(0.5)
    z_steps = y_steps = 200
    Rz = Rotation.from_euler('ZYX', [alpha, 0, 0, ]).as_matrix()
    Rzy = Rotation.from_euler('ZYX', [alpha, beta, 0, ]).as_matrix()
    dRz = Rotation.from_euler('ZYX', [alpha/z_steps, 0, 0, ]).as_matrix()
    dRy = Rotation.from_euler('ZYX', [0, beta/y_steps, 0]).as_matrix()
    dRx = Rotation.from_euler('ZYX', [0, 0, del_gamma]).as_matrix()

    print("Rotating along z axis by -45 deg")
    for _ in range(z_steps):
        pcd.rotate(dRz, pcd.get_center())
        axis.rotate(dRz, axis.get_center())
        vis.update_geometry(pcd)
        vis.update_geometry(axis)
        vis.poll_events()
        vis.update_renderer()
        sleep(1/60)
    sleep(0.5)

    print("Rotating about Y axis by 90 deg")
    for _ in range(y_steps):
        rot_mat = Rz@dRy@Rz.T
        pcd.rotate(rot_mat, pcd.get_center())
        axis.rotate(rot_mat, axis.get_center())
        vis.update_geometry(pcd)
        vis.update_geometry(axis)
        vis.poll_events()
        vis.update_renderer()
        sleep(1/60)
    sleep(0.5)

    print("Rotating about X axis")
    for _ in range(360):
        rot_mat = Rzy@dRx@Rzy.T
        pcd = pcd.rotate(rot_mat, pcd.get_center())
        axis.rotate(rot_mat, axis.get_center())
        sleep(1/60)
        vis.update_geometry(pcd)
        vis.update_geometry(axis)
        vis.poll_events()
        vis.update_renderer()
    sleep(0.8)



In [5]:
print("Task 2.a.1")
task_2_a_1()
print()
print()
print("Task 2.a.2")
task_2_a_2()

Task 2.a.1
The Calculated Rotation Matrix for [ 0 ,  0 ,  1.5707963267948966 ] is :
[[ 1.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  6.123234e-17 -1.000000e+00]
 [ 0.000000e+00  1.000000e+00  6.123234e-17]]
Real Rotation matrix:
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  2.22044605e-16 -1.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  2.22044605e-16]]


Task 2.a.2

 for M: 
[[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542533  0.14007684]
 [-0.93969262  0.17101007  0.29619813]] 


The  input matrix (alpha,beta,gamma) is : 
 [[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542533  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]

 Initialization 1 is :  [0.5488135  0.71518937 0.60276338] 
 Result obtained is :  [0.6981317  1.22173048 0.52359878] 


 Initialization 2 is :  [0.54488318 0.4236548  0.64589411] 
 Result obtained is :  [0.6981317  1.22173048 0.52359878] 


 Initialization 3 is :  [0.43758721 0.891773   0.963662

QUESTION 2.a.3: What is Gimbal Lock?  
ANSWER:  
A gimbal is a hanging ring that can revolve about one axis. When two axes coincide, a DOF is lost, which may be devastating for control systems. To allow rotation about several axes, gimbals are generally stacked one within another.
Gimbal lock occurs when the innermost ring becomes aligned with the outermost ring during three-axis Euler rotations. This is simply achieved by rotating the mid-ring 90 degrees. As a result, an axis of rotation is lost since two of the axes are now precisely the same. This can leads to odd behavior while rotating. Due to the characteristics of covering spaces, gimbal lock will definitely occur at some point in the system for situations of three or fewer stacked gimbals. There is no gimbal restraint. All three gimbals may still freely revolve about their respective suspension axes. However, because of the parallel alignment of two of the axes of the gimbals, there is no gimbal is available to allow rotation along one axis.


QUESTION 2.a.4: Show an example where a Gimbal lock occurs and visualize the Gimbal lock.  

Answer:  
The loss of 1 dof is an issue with the gimbal lock and it is the main problem with the Euler representations. So Gimbal Lock might occur in systems that use Euler angles. The following are some of the systems: Accelerometers, gyroscopes, robotic wrists, gunfire control systems.
A better example can be given by referring to the definition of rotation matrix from JJ Craig book and using Euler angles ZYX, for (a,90,c), we get a rotation matrix  as  
$$R(a , 90 ,c)=\left[\begin{array}{rrr}0 & sin(c - a) & cos(c-a) \\0 & cos(c-a) & -sin(c-a) \\
    -1 & 0 & 0\end{array}\right] 
$$
The formula is now left with effectively one variable c-a .Hence, we can see there is only one degree of freedom left.

In [8]:
task_2_a_4()

Rotating along z axis by -45 deg
Rotating about Y axis by 90 deg
Rotating about X axis


## b) Quaternions

1. What makes Quaternions popular in graphics? 
2. Convert a rotation matrix to quaternion and vice versa. Do not use inbuilt libraries for this question.
3. Perform matrix multiplication of two $\mathcal{R}_{3 \times 3}$ rotation matrices and perform the same transformation in the quaternion space. Verify if the final transformation obtained in both the cases are the same.
4. Try to interpolate any 3D model (cube / bunny / not sphere obviously!!) between two rotation matrices and visualize!

The above questions require you to **code your own functions** and **only verify** using inbuilt functions.

In [2]:
M = np.array([
    [0.26200263, -0.19674724, 0.944799],
    [0.21984631, 0.96542533, 0.14007684],
    [-0.93969262, 0.17101007, 0.29619813]])


def mat_to_quat(M):
    q = np.zeros((4,))
    tr = np.trace(M)

    if math.isclose(tr, 3):
        q[0] = 1

    elif math.isclose(tr, 1):
        q[0] = 0
        q[1] = math.sqrt((M[0][0]+1)/2)
        q[2] = math.sqrt((M[1][1]+1)/2)
        q[3] = math.sqrt((M[2][2]+1)/2)
    else:
        theta = math.acos((tr-1)/2)
        rx = M[2][1] - M[1][2]
        ry = M[0][2] - M[2][0]
        rz = M[1][0] - M[0][1]
        q[0] = math.cos(theta/2)
        q[1:] = (math.sin(theta/2)/(2*math.sin(theta)))*np.array([rx, ry, rz])
    return q


def quat_to_mat(q):
    w, x, y, z = q
    M = np.array([
        [1 - 2*(y*y + z*z), 2*(x*y-z*w), 2*(x*z + y*w)],
        [2*(x*y + z*w), 1 - 2*(x*x + z*z), 2*(y*z - x*w)],
        [2*(x*z - y*w), 2*(y*z+x*w), 1 - 2*(x*x + y*y)]
    ])
    return M


def real_Mq(M):
    r = R.from_matrix(M)
    a = r.as_quat()
    return a[[3, 0, 1, 2]]


def real_qM(q):
    q = q[[1, 2, 3, 0]]
    a = R.from_quat(q).as_matrix()
    return a


def get_random_rot_matrix():
    return R.from_quat(np.random.rand(4)).as_matrix()


def get_random_quat():
    return R.from_quat(np.random.rand(4)).as_quat()[[3, 0, 1, 2]]


def task_2_b_2():
    M = get_random_rot_matrix()
    print("Real Matrix")
    print(M)
    print()

    print("calculated quaternion")
    print(mat_to_quat(M))
    print()
    print("Expected quaternion")
    print(real_Mq(M))
    print()

    print("====================")
    q = get_random_quat()
    print("Real quaternion")
    print(q)
    print()
    print("expected matrix")
    print(quat_to_mat(q))
    print()
    print("real matrix")
    print(real_qM(q))
    print()


def multiply_quats(q1, q2):
    p0, p = q1[0], q1[1:]
    q0, q = q2[0], q2[1:]
    res = np.zeros((4,))
    res[0] = p0*q0 - np.dot(p, q)
    res[1:] = p0*q + q0*p + np.cross(p, q)
    return res


def my_to_s(q):
    return np.array(q)[[1, 2, 3, 0]]


def s_to_my(q):
    return np.array(q)[[3, 0, 1, 2]]


def task_2_b_3():
    np.random.seed(1)
    R1 = get_random_rot_matrix()
    R2 = get_random_rot_matrix()
    final_R = R1@R2

    q1 = mat_to_quat(R1)
    q2 = mat_to_quat(R2)
    final_q = multiply_quats(q1, q2)

    print("calculated Matrix from matrix multiplication")
    print(final_R)
    print()
    print("calculated Quaternion from quaternion multiplication")
    print(final_q)
    print()

    r1 = R.from_matrix(R1)
    r2 = R.from_matrix(R2)
    r = r1*r2
    print("Real Matrix from matrix multiplication")
    print(r.as_matrix())
    print("Real Quaternion from quaternion multiplication")
    # print(mat_to_quat(r.as_matrix()))
    print(r.as_quat()[[3, 0, 1, 2]])


def task_2_b_4():
    # np.random.seed(0)
    def do_task():
        R1_theta = np.pi*np.zeros(3)
        R2_theta = np.pi*np.array([1,1,0.])
        num_steps = 300
        del_theta = R2_theta/num_steps
        R1 = o3d.geometry.get_rotation_matrix_from_zyx(R1_theta)
        dR = o3d.geometry.get_rotation_matrix_from_zyx(del_theta)

        pcd = Pointcloud().pcd
        axis = get_axis(0.05, [0, 0, 0])
        pcd = pcd.rotate(R1, pcd.get_center())
        axis = axis.rotate(R1, axis.get_center())

        vis = o3d.visualization.Visualizer()
        vis.create_window()
        vis.add_geometry(deepcopy(axis))
        axis.translate([0.1, 0, 0])
        vis.add_geometry(axis)
        vis.add_geometry(pcd)
        vis.poll_events()
        vis.update_renderer()
        sleep(0.5)

        for _ in range(num_steps):
            pcd = pcd.rotate(dR, pcd.get_center())
            axis = axis.rotate(dR, axis.get_center())
            vis.update_geometry(pcd)
            vis.update_geometry(axis)
            vis.poll_events()
            vis.update_renderer()
            sleep(1/60)
        sleep(1)

    for _ in range(3):
        do_task()



QUESTION 2.b.1: What makes Quaternions popular in graphics?  
ANSWER: Quaternions are popular because:  
- these do not suffer from Gimbal Lock because only one rotation is there.
- Quaternions consume less memory and are faster to compute than matrices. 
- To recover Quaternions from rotation matrix, we do not get complex trancedental trignometric equations.

In [8]:
print("Task 2.b.2")
task_2_b_2()

Task 2.b.2
Real Matrix
[[-0.31742274  0.00456944  0.94827313]
 [ 0.87776432  0.37983068  0.2919905 ]
 [-0.358849    0.92504474 -0.12457779]]

calculated quaternion
[0.48420816 0.32685026 0.67487614 0.45083652]

Expected quaternion
[0.48420816 0.32685026 0.67487614 0.45083652]

Real quaternion
[0.02167919 0.99247179 0.07616847 0.09342447]

expected matrix
[[ 0.97094047  0.14713939  0.18874484]
 [ 0.15524086 -0.98745675 -0.02879998]
 [ 0.18213975  0.05726397 -0.98160376]]

real matrix
[[ 0.97094047  0.14713939  0.18874484]
 [ 0.15524086 -0.98745675 -0.02879998]
 [ 0.18213975  0.05726397 -0.98160376]]



In [9]:
task_2_b_3()

calculated Matrix from matrix multiplication
[[ 0.44899509  0.8864049  -0.11264885]
 [ 0.89339678 -0.443133    0.07399553]
 [ 0.01567158 -0.13386375 -0.99087582]]

calculated Quaternion from quaternion multiplication
[-0.06120921  0.84897054  0.52410584 -0.02855731]

Real Matrix from matrix multiplication
[[ 0.44899509  0.8864049  -0.11264885]
 [ 0.89339678 -0.443133    0.07399553]
 [ 0.01567158 -0.13386375 -0.99087582]]
Real Quaternion from quaternion multiplication
[-0.06120921  0.84897054  0.52410584 -0.02855731]


In [6]:
task_2_b_4()

## c) Exponential maps (Bonus)

1. What is the idea behind exponential map representation of rotation matrices?
2. Perform matrix exponentiation and obtain the rotation matrix to rotate a vector $P$ around $\omega$ for $\theta$ seconds.
$$
\omega = \begin{bmatrix}2 \\ 1 \\ 15 \end{bmatrix}
$$

$$
\theta = 4.1364
$$

3. Compute the logarithmic map (SO(3) to so(3)) of the rotation matrix to obtain the rotation vector and the angle of rotation
$$
\begin{bmatrix}
0.1 &  -0.9487 & 0.3 \\
0.9487 & 0.  & -0.3162 \\
0.3   &  0.3162  & 0.9 
\end{bmatrix}
$$
You can use inbuilt libraries **only to verify** your results.

IDEA BEHIND EXPONENTIAL MAP:  
Any rotation can be considered as a rotation along an axis with some angle theta. For any given time, the rate of change of any vector, when being rotated by a rotation is given by  w X v, where v is the vector and w is the three-parameter orientation representation. These three parameters are the exponential coordinates, which describe the direction of one frame in relation to another. Because of the link to linear differential equations, we call this another form of a rotation matrix exponential coordinates.

THe exponential map is a map which converts a vector to its rotated form.

<img src="misc/a.jpeg" alt="drawing" width="500"/>  
<img src="misc/b.jpeg" alt="drawing" width="500"/>

In [11]:
def rotation_matrix(omega, theta):
    I = np.identity(3)
    omegahat = np.array(
        [[0, -omega[2], omega[1]], [omega[2], 0, -omega[0]], [-omega[1], omega[0], 0]])
    # print(omegahat)
    omagahatsq = np.dot(omegahat, omegahat)
    # print(omagahatsq)
    term2 = np.multiply(np.sin(theta), omegahat)
    term3 = np.multiply((1-np.cos(theta)), omagahatsq)
    # print(term3)
    R = np.add(term2, term3)
    # print(R)
    R = np.add(R, I)
    return R

def log_map(m):

    tr=m[0][0]+m[1][1]+m[2][2]
    if (tr == -1):
        theta=np.pi
        if(m[2][2]!=-1 ):
            omegahat=np.array([m[0][2],m[1][2],1+m[2][2]])
            multiplying_factor=1/np.sqrt(2*(1+m[2][2]))
            omegahat=np.multiply(multiplying_factor,omegahat)
            print("\ntheta for the given matrix is : ",theta)
            print("\nOmega for the given matrix is : \n",omegahat)

        elif(m[1][1]!=-1):
            omegahat=np.array([m[0][1],1+m[1][1],m[2][1]])
            multiplying_factor=1/np.sqrt(2*(1+m[1][1]))
            omegahat=np.multiply(multiplying_factor,omegahat)
            print("\ntheta for the given matrix is : ",theta)
            print("\nOmega for the given matrix is : \n",omegahat)
           
        else:
            omegahat=np.array([1+m[0][0],m[1][0],m[2][0]])
            multiplying_factor=1/np.sqrt(2*(1+m[0][0]))
            omegahat=np.multiply(multiplying_factor,omegahat)
            print("\ntheta for the given matrix is : ",theta)
            print("\nOmega for the given matrix is : \n",omegahat)
       

    elif(m==np.identity(3)).all():
        theta=0
        print("Omega is not defined for this case")
        omegahat=np.array([])
        print("\ntheta for the given matrix is : ",theta)

    
    else:
        theta=np.arccos(0.5*(tr-1))
        multiplying_factor=1/(2*np.sin(theta))
        omegahat=np.subtract(np.array(m),np.array(m).transpose())
        omegahat=np.multiply(multiplying_factor,omegahat)
        z,y,x=-1*omegahat[0][1],omegahat[0][2],omegahat[2][1]
        print("\ntheta for the given matrix is : ",theta)
        print("\nOmegahat for the given matrix is : \n",omegahat)
        print("\nGetting omega from the above omegahat ....")
        print("\nOmega for the given matrix is : ",[x,y,z])

def task_2_c_2():
    omega = np.array([2, 1, 15])
    theta = 4.1364
    print("Rotation matrix for omega : ", omega,
          " and theta : ", theta, " is :")
    R = rotation_matrix(omega, theta)
    print("\n", R)

def task_2_c_3():
    m = np.array(
        [[0.1, -0.9487, 0.3], [0.9487, 0., -0.3162], [0.3, 0.3162, 0.9]])
    # m=np.array([[-1,-0.9487,0.3],[0.9487,1,-0.3162],[0.3,0.3162,-1]])
    # m=np.eye(3)
    log_map(m)


In [12]:
task_2_c_2()
task_2_c_3()

Rotation matrix for omega :  [ 2  1 15]  and theta :  4.1364  is :

 [[-348.09417007   15.66913969   45.50128003]
 [  -9.49048181 -352.72816347   24.84727514]
 [  47.17858813   21.49265894   -6.72332235]]

theta for the given matrix is :  1.5707963267948966

Omegahat for the given matrix is : 
 [[ 0.     -0.9487  0.    ]
 [ 0.9487  0.     -0.3162]
 [ 0.      0.3162  0.    ]]

Getting omega from the above omegahat ....

Omega for the given matrix is :  [0.3162, 0.0, 0.9487]


# 3. Data representations

## a) Octomaps

1. Why is an Octomap memory efficient?
2. When do we update an Octomap and why?
3. When would you likely use an octomap instead of a point cloud?
 

ANSWER 3.a.1  
Octomap implements 3D occupancy grid mapping, which provides data structure and mapping algorithms in C++ that are especially suitable for robotics. The implementation of the map is based on octree.
Octomap (by default) does not store individual points but instead, it integrates them into a maximum compression occupancy map, in order to decrease memory consumption.
It utilizes an octree-based data structure for probabilistic data collection while simultaneously compressing the storage necessary to only a couple of bits per child node while retaining the distinction between occupied, unoccupied, and unknown cells, therefore, reducing the memory.
(Simply it can be put as: we expand only that section in which we are finding partial data into 8 parts. We are not making child for those which are completely occupied or completely empty and this reduces the storage.)


ANSWER 3.a.2:  
With additional sensor readings, we can continually update the map but the original point clouds
are not stored  (but can be stored). New data or sensor readings can be added at any moment.
Modeling and updating are done in a probabilistic manner. This results in the sensor noise or measures 
that arise from dynamic environmental changes, for example, due to dynamic objects. 
Moreover, several robots can contribute to the same map, and while exploring new places, 
and earlier map can be expanded. Whenever we find a partially occupied cell, we just explore 
and update that node for minimizing storage and for getting finer details about the environment or node.


ANSWER 3a.3  
OctoMap is an integrated Octree-based open-source framework that is well known for its memory 
efficiency for the representation of 3D environments. 
Point cloud cannot be applied in robot navigation, instead, OctoMap can be used as OctoMap is 
a 3D occupancy grid mapping and can be applied to 3D path planning of flying robots and different 
robots that are outfitted with manipulators.

## b) Signed Distance Functions

1. How do we determine object surfaces using SDF?
2. How do we aggregate views from multiple cameras? (just a general overview is fine)
3. Which preserves details better? Voxels or SDF? Why?
4. What’s an advantage of SDF over a point cloud?


ANSWER 3.b.1:   
Signed-distance functions (SDF), when passed the coordinates of a point in space, return the shortest distance between that point and some surface. The sign of the return value indicates whether the point is inside (negative value returned) that surface or outside (positive value returned) and on the surface (when zero value is returned). So if the value returned is zero, it means the point is on the surface.

ANSWER 3.b.2:  
When we have views from two cameras, we get the corresponding points in both images. Corresponding points are the points common to both images. For this, we use equipolar geometry. Epipolar geometry is the geometry of stereo vision. When two cameras observe a 3D scene from two different places, a number of geometric interplays occur between the 3D points and the 2D pictures that lead to limitations between them. The main thing is getting the common points from pair of images , then we can do image stitching or 3D reconstruction etc by correlating points.  

ANSWER 3.b.3:  
SDFs can preserve the details better. Accurate weighing based on measurement confidence can help us better preserve details. In OctoMap all measurements are given same weightage. SDFs store more information than voxels, like distance measure and confidence weights.

ANSWER 3.b.4:
SDF uses far less memory thn point clouds since only the voxels closer to the surface need to be considered.
Apart from that, point clouds only store occupancy probability, but SDFs stores weights and distances from the surface, which can help us to determine a voxel's confidence and position wrt to the surface (inside or outside). Also since the distances are directly stored in SDFs, we can use them in trajectory generation.

# References and Resources

1. Gimbal locks and quaternions: https://youtu.be/YF5ZUlKxSgE
2. Exponential map: 
    1. 3 Blue 1 Brown: https://youtu.be/O85OWBJ2ayo
    2. Northwestern Robotics: https://youtu.be/v_KBHaG0mas
3. Bunny ply is taken from: http://graphics.im.ntu.edu.tw/~robin/courses/cg03/model/